# Urban Traffic Anomaly Detection using Variational Autoencoders (VAE)
## Using VAE to build a Traffic Anomaly Detector for Early Accident Warnings

In this project, we aim to detect anomalies in urban traffic patterns using a probabilistic machine learning approach based on Variational Autoencoders (VAE). 

We will train the VAE on typical traffic data so that it learns common patterns. Then, we will detect anomalies as deviations from these learned patterns.

The dataset used is the METR-LA dataset, which contains traffic speed readings from sensors in Los Angeles collected every 5 minutes.


---
## **Outline**
### **Goal**

>Build a system that spots unusual traffic patterns in real time and warns authorities of possible upcoming accidents.


### **How It Works, Simply: “Normal” vs “Strange”**:

   * We first teach a model what everyday traffic looks like (no crashes OR really unbalanced dataset with low incidents/traffic-recordings ratio <span style="color:red">{*we suppose our dataset to be really unbalanced, as they do in a paper*}</span> ).
   * Then, in operation, the model watches incoming traffic data and flags anything that “doesn’t fit” its learned normal patterns.

1. **Data in Chunks**

   * preprocessing 
      * missing data imputation, if any.
      * standardization only on training data, or on test data too  (???????????????????????????
      * NORMAL TRAFFIC DATA SHOW TIME PATTERNS EVERYTIME EVEN IN REGULAR CONDITIONS, HOW TO ADDRESS THIS THING \
      <span style="color:red">{*should a column with categorical number of week day be added to keep track of regular patterns in time? What about long period patterns, as monthly and yearly patterns??????????????????????????????*}</span> 
   * We group sensor readings into short time blocks (e.g. one hour of readings) \
   <span style="color:red">{*could the split be made by letting in and letting out a single timestamped row at a time?....for sure it is COMPUTATIONALLY DEMANDINg, any advantage THEN?*}</span> 

2. **Model Magic (Under the Hood)**

   * A compact “autoencoder” network learns to compress each block into a small summary.... 
      * build the model (which latent sizes, layer widths, KL-weight schedules, etc....)
      * maximize ELBO to optimize the parameters (see below theory)
      * test (cross validation/others on a validation set (e.g. last 10% of days) )
   * <span style="color:red">*TRY DIFFERENT VAE VERSIONS....LOOKING AT LITERATURE RIGHT NOW*</span> 


3. **Flagging Early Warnings**
   * ....and then reconstruct it back. If it reconstructs poorly, that block is probably unusual.
   * We check reconstruction error: big errors → possible problem.
   * When multiple sensors or blocks light up before a known crash, we call those “early warnings.”


## **About 'Flagging Early Warnings' (TEMPORARY DRAFT)**

Once our model has learned what “normal” traffic looks like, here’s how we catch potential accidents before they happen:

1. **Watch the Reconstruction Error**

   * **What it is**: For each time block and each sensor, the model tries to rebuild (reconstruct) the recorded speeds. If the rebuilt value is very different from the real one, that’s an error.
   * **Why it matters**: A low error means “all good,” a high error means “something strange is going on here.”
   * **Compute a Simple Anomaly Score**

    * For each sensor in a block, calculate specific sensor’s reconstruction error across the multiple rows of the block. Could be the MSE??????

      $$
        \mathrm{Error}_s \;=\;\sum_{t\in\text{block}}(\text{actual}_{t,s}-\hat{\text{predicted}}_{t,s})^2
      $$


2. **Ways to Flag Glitches** 
    * **2.1 Per-Sensor Alerts (Instant)**

      * **What:** As soon as any sensor’s error > its own cutoff (e.g. 99ᵗʰ-percentile), you send a “glitch” alert for that sensor.
      * **Why:** Super fast, catches tiny/local faults immediately.
      * **Trade-off:** Every alert looks equally urgent and you might get noise.

    * **2.2 Add a Quick Severity Score (Prioritized)**

      * **Measure:**

        1. **MaxErrorRatio** = worst sensor’s error ÷ its normal 95ᵗʰ-percentile error
        2. **CountFired** = how many sensors are over their cutoffs
        3. **SeverityScore** = MaxErrorRatio × √CountFired

      * **Map to Low/Med/High:**

        * Low if < 2 • Medium if 2–5 • High if ≥ 5
      * **Why:** Highlights big or widespread glitches, cuts noise, still trivial math.
    
    * **2.3 Set Smart Thresholds**
      * Summarize across sensors, e.g.:

          $$
            \text{BlockScore} = \max_s \; \text{Error}_{s}
          $$
      * Intuition: if even one sensor suddenly behaves oddly, the block gets flagged.
      * Look at BlockScores over a week of known “normal” days.
      * Choose a cutoff (e.g. the 95ᵗʰ percentile) so that only the largest 5% of errors trigger an alert.
      * This keeps false alarms low while still catching real issues.
      


 


4. **Detecting Early Warnings vs. Aftermath**

   * **Early Warning**: If a high BlockScore occurs **before** any reported incident, mark it as a “precursor.”
   * **Aftermath**: If it happens **during or immediately after** a crash, label it “consequence.”
   * We’ll visualize these on a timeline—warning flags should lead incidents by several minutes if truly predictive.

5. **Optional Classifier for Precision**

   * Feed the model’s internal summary (its compact “code” for each block) into a tiny decision tree or logistic regression.
   * Train it on a handful of labeled examples (blocks known to be “warnings” vs. “consequences”). \
   <span style="color:red">{*just know two datasets needs to be jointed and a categorical variable eventually added. (???). We also have a dataset with coordinates of incidents, and our traffic dataset has references to coordinates too*}</span>
   * This helps the system learn subtle patterns that pure error thresholds might miss (e.g. gradual slowdowns vs. sudden stops).

6. **(Ideal) Real-Time Alerting Flow**

   1. New block arrives every 5 min.
   2. Model computes BlockScore on the fly.
   3. If BlockScore > threshold → flag an alert.
   4. Check classifier (if used) to tag it “precursor” or “aftermath.”
   5. Push notification to dashboard and/or SMS/email to operators.


This step-by-step approach keeps things transparent—each alert is simply “error too big,” optionally refined by a lightweight classifier, and timed relative to known incidents to tell if it really is an early warning.




## **Do We Have Enough Data?**

* **Learning “normal” traffic**: \~2–4 weeks of continuous data at 5 min intervals (i.e. 400–800 windows) usually suffices to cover daily and weekly patterns.
* **Distinguishing precursors vs. consequences**: Aim for **50–100 past incidents** to reliably calibrate thresholds and/or train the simple classifier.

  * If you see \~1 incident per day, 2–3 months of data will do; if incidents occur \~1 per week, plan for 1–2 years of history.

In short: modeling normal behavior is quick, but confidently tagging early warnings requires a few dozen labeled incidents.


## **3-Week Roadmap (TEMPORARY DRAFT)**

| Week  | Focus & Deliverables                                                                                                                                                                                                                                                                                                                |
| ----- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1** | **VAE model** <br>• Preprocessing on our traffic data.<br>• Add day-of-week and hour-of-day so the model knows “rush hour vs. quiet time.” <span style="color:red">{*?????not sure ?????*}</span> <br>• Build a simple model that learns “normal” patterns. <br>• Test it. <br>• Calibrate the “error threshold” so we catch real incidents but avoid too many false alarms.  <span style="color:red">{*How to compute each sensor’s 95ᵗʰ-percentile error?*}</span>                           |                                                                           |
| **2** | **Early-Warning Classifier & Demo**<br>• Either build per-sensor alerts AND/OR add SeverityScore formula & pick cutoffs. <br> • or Mark blocks before known accidents as “warnings” and others as “after” or “normal.”<br>• Train a small classifier on the model’s summary outputs to tell “warning vs. aftermath.”<br>• Put together a simple live dashboard that shows sensor map, current alerts, and warning lead times. |
| **4** | **(Optional) real time pipeline**<br>• Hook into real-time pipeline (every 5 min)<br>• Dashboard shows sensor flags + severity |

### **Ideal goal** (again)

> Real time system to flag anomalies and pop out early-warnings

---

## **Environment Setup**

We will use Python 3.8+ and the following key libraries:

- `h5py`: to read `.h5` dataset files  
- `tables` (PyTables): a dependency needed by `pandas` to handle HDF5 files  
- `numpy` and `pandas`: for data manipulation  
- `matplotlib`: for visualization  
- `torch` (PyTorch): to build and train our VAE model  

### Installing required packages

You can install them via pip:

```bash
pip install numpy pandas matplotlib h5py tables torch


## **Data Description**


**METR-LA Dataset**

- Contains traffic speed data from 207 sensors in Los Angeles.
- Data is recorded every 5 minutes, resulting in 12 records per hour.
- The data is stored in an HDF5 file format (`metr-la.h5`), where rows represent timestamps and columns correspond to different sensors.
- Additionally, a precomputed sensor graph adjacency matrix is provided (`adj_mx.pkl`) which encodes the spatial relations between sensors.

The data shape is approximately (34272, 207), meaning 34,272 time steps and 207 sensors.


## Data Loading and Preprocessing

We will load the data from the `.h5` file using `h5py`, normalize the data, and create sliding windows of size 12 (1 hour of data).

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ruta al archivo .h5
file_path = 'data/metr-la.h5'
df = pd.read_hdf(file_path)
print(df.shape)
print(df.head)

(34272, 207)
<bound method NDFrame.head of                         773869     767541     767542     717447     717446  \
2012-03-01 00:00:00  64.375000  67.625000  67.125000  61.500000  66.875000   
2012-03-01 00:05:00  62.666667  68.555556  65.444444  62.444444  64.444444   
2012-03-01 00:10:00  64.000000  63.750000  60.000000  59.000000  66.500000   
2012-03-01 00:15:00   0.000000   0.000000   0.000000   0.000000   0.000000   
2012-03-01 00:20:00   0.000000   0.000000   0.000000   0.000000   0.000000   
...                        ...        ...        ...        ...        ...   
2012-06-27 23:35:00  65.000000  65.888889  68.555556  61.666667   0.000000   
2012-06-27 23:40:00  61.375000  65.625000  66.500000  62.750000   0.000000   
2012-06-27 23:45:00  67.000000  59.666667  69.555556  61.000000   0.000000   
2012-06-27 23:50:00  66.750000  62.250000  66.000000  59.625000   0.000000   
2012-06-27 23:55:00  65.111111  66.888889  66.777778  61.222222   0.000000   

                    

#### Data Normalization

We use `StandardScaler` from scikit-learn to apply Z-score normalization to the dataset.

Each sensor (column) is standardized independently by removing the mean and scaling to unit variance:

$$
x_{\text{normalized}} = \frac{x - \mu}{\sigma}
$$

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_normalized = scaler.fit_transform(df.values)  # shape: (34272, 207)


#### Sliding Windows

Sliding windows allow us to structure the time series data as sequences of fixed length, suitable for feeding into our VAE model.

The window size of 12 means each sample contains 12 consecutive timesteps (i.e., one hour of data).


In [ ]:
def create_sliding_windows(data, window_size):
    X = []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
    return np.array(X)

window_size = 12  # 12 pasos = 1 hora
X = create_sliding_windows(data_normalized, window_size)  # shape: (34260, 12, 207)
print(X.shape)


(34260, 12, 207)


#### Data splitting

In [ ]:
import numpy as np

# X.shape = (34260, 12, 207)
num_samples = X.shape[0]

# Definimos proporciones
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculamos índices
train_end = int(num_samples * train_ratio)
val_end = train_end + int(num_samples * val_ratio)

# División
X_train = X[:train_end]
X_val = X[train_end:val_end]
X_test = X[val_end:]

print(f'Train shape: {X_train.shape}')
print(f'Validation shape: {X_val.shape}')
print(f'Test shape: {X_test.shape}')


Train shape: (23982, 12, 207)
Validation shape: (5139, 12, 207)
Test shape: (5139, 12, 207)


## Variational Autoencoder (VAE)

A Variational Autoencoder (VAE) is a type of generative model that learns to encode input data into a compressed latent space and then reconstruct it as accurately as possible. Unlike traditional autoencoders, VAEs add a probabilistic framework that makes the latent space continuous and meaningful, which is useful for tasks like anomaly detection.

### Main Components

#### 1. **Encoder**
The encoder network takes an input (e.g., a time window of traffic data) and maps it to a latent distribution. Instead of directly outputting a single latent vector, the encoder outputs:

- A mean vector $\mu$
- A standard deviation (or log variance) vector $\log(\sigma^2)$

These define a multivariate normal distribution from which we sample the latent variable $z$.

#### 2. **Latent Sampling**
To allow backpropagation through the sampling process, VAEs use the **reparameterization trick**:

$$
z = \mu + \sigma \cdot \epsilon \quad \text{where} \quad \epsilon \sim \mathcal{N}(0, I)
$$

This makes the sampling step differentiable.

#### 3. **Decoder**
The decoder takes the sampled latent variable $z$ and tries to reconstruct the original input data. The goal is to learn a meaningful latent space that can generate realistic reconstructions.

### Loss Function

The VAE optimizes two components in the loss:

1. **Reconstruction Loss** (e.g., Mean Squared Error):

$$
\mathcal{L}_{\text{recon}} = ||x - \hat{x}||^2
$$

This penalizes the model when its reconstruction $\hat{x}$ is far from the original input $x$.

> **Why use MSE as reconstruction loss?**
>
> Theoretically, we want to maximize the log-likelihood of the reconstruction, i.e., $\log p(x|z)$. If we assume that the decoder’s output is Gaussian with fixed variance (i.e., $p(x|z) = \mathcal{N}(\hat{x}, \sigma^2 I)$), then:
>
> $$
> \log p(x|z) = -\frac{1}{2\sigma^2} ||x - \hat{x}||^2 + \text{const}
> $$
>
> So minimizing the squared error (MSE) is equivalent to maximizing the likelihood. That’s why MSE acts as a **proxy** for the negative log-likelihood in practice.


2. **Kullback-Leibler (KL) Divergence**:

$$
\mathcal{L}_{\text{KL}} = D_{\text{KL}}(q(z|x) || p(z))
$$

This measures how much the learned latent distribution $q(z|x)$ (given by the encoder) deviates from the standard normal prior $p(z) = \mathcal{N}(0, I)$.

> **In practice**, the encoder outputs $\mu$ and $\log(\sigma^2)$, and the KL divergence between the approximate posterior and the prior can be computed in closed form:
>
> $$
> \mathcal{L}_{\text{KL}} = -\frac{1}{2} \sum \left(1 + \log(\sigma^2) - \mu^2 - \sigma^2\right)
> $$
>
> This encourages the model to keep the latent representations close to a standard normal distribution.


### Total Loss

The total loss combines both terms:

$$
\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{recon}} + \beta \cdot \mathcal{L}_{\text{KL}}
$$

where $\beta$ is a hyperparameter (commonly set to 1) that balances reconstruction accuracy and regularization. 

> **Important:** In theory, we want to **maximize** the Evidence Lower Bound (ELBO), which is:
>
> $$
> \text{ELBO}(x) = \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{\text{KL}}(q(z|x) || p(z))
> $$
>
> But in practice, optimization libraries **minimize** functions. So minimizing the total loss is equivalent to **maximizing the ELBO**.